In [ ]:
import utils.data as ud
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

In [ ]:
import statsmodels.api as sm

In [ ]:
import tbtools.dev as tbdev
import tbtools.panda as tbpd
import tbtools.iter as tbiter
import tbtools.strings as tbstr

In [ ]:
import utils.evaluation.modelling as uem

## Assign train, validate, test labels

In [ ]:
from sklearn.cross_validation import train_test_split

import utils.features as uf

In [ ]:
uf = tbdev.reload(uf)

In [ ]:
# %%time
# x,y = uf.get_x_y( c_lags=['{} min'.format(L) \
#                           for L in (
#                                         5,
#                                         10,
#                                         20,
#                                         30,
#                                     )
#                           ],
#                   dc_settings=[
#                     ('{} min'.format(L), '{} min'.format(W), sign) \
#                         for L in (
#                                     5,
#                                     10,
#                                     20,
#                                     30,
#                                  ) \
#                         for W in (
#                                     1,
#                                     2,
#                                     5,
#                                     10,de
#                                  ) \
#                         for sign in (
#                                     '+',
#                                     '-',
#                                     '',
#                                     )
#                                 ]
#                 )

# tbdev.notify('Finished fetching features')

In [ ]:
# x['C'] = y

# _train, _test = train_test_split(x, test_size=0.2, random_state=0)
# _train, _validate = train_test_split(_train, test_size=.25, random_state=1)

In [ ]:
# def xy(v):
#     return v.drop('C', axis=1), v['C']

# x_train, y_train = xy(_train)
# x_validate, y_validate = xy(_validate)
# x_test, y_test = xy(_test)

In [ ]:
# def store_designmatrices():
#     import os
#     p = ud.paths.Paths.design_matrices
#     dmpath = lambda x: os.path.join(p, 'c_dc_' + x)

#     x_train.to_pickle(dmpath('x_train.pickle'))
#     y_train.to_pickle(dmpath('y_train.pickle'))
#     x_validate.to_pickle(dmpath('x_validate.pickle'))
#     y_validate.to_pickle(dmpath('y_validate.pickle'))
#     x_test.to_pickle(dmpath('x_test.pickle'))
#     y_test.to_pickle(dmpath('y_test.pickle'))

In [ ]:
def dmpath(p):
    import os
    dm = ud.Paths.design_matrices
    return os.path.join(dm, 'c_dc_' + p)

x_train = pd.read_pickle(dmpath('x_train.pickle'))
y_train = pd.read_pickle(dmpath('y_train.pickle'))
# x_validate = pd.read_pickle(dmpath('x_validate.pickle'))
# y_validate = pd.read_pickle(dmpath('y_validate.pickle'))
# x_test = pd.read_pickle(dmpath('x_test.pickle'))
# y_test = pd.read_pickle(dmpath('y_test.pickle'))

In [ ]:
fig = sns.plt.figure()
sns.plt.plot((x.index - x.index.min()).total_seconds()[::50])

In [ ]:
x_train.shape#, x_validate.shape, x_test.shape

---

My target variable is definitely not normal.

In [ ]:
sm.qqplot(y_train, line='q');

In [ ]:
# No boxcox can fix this target :-/
from scipy.stats import boxcox
sm.qqplot(boxcox(y_train+2)[0], line='q');

# Plot variables against target

In [ ]:
# Sort first by feature name, then window size, and the lag length
cols = tbstr.sort_strings(x_train.columns, ['^\S+', 'W=(\d+)', 'L=(\d+)'])

In [ ]:
n_cols = 4
n_rows = (x_train.shape[1] + 1) // n_cols
fig, axs = sns.plt.subplots(n_rows, n_cols, figsize=(15, n_rows*3.5))
axs = axs.flatten()
    
for ax, c in zip(axs, cols):
    tbpd.hist2d(y_train, x_train[c], ax=ax,
                integer_aligned_bins=True, sqrt=True);
fig.tight_layout()
tbdev.notify('Finished drawing variables')

---

anova to compare models?

F-test to compare models?

log-likelihood

In [ ]:
import utils.evaluation.modelling as uem

In [ ]:
# TIMES = ('day', 'hour', 'minute')

# def rmse(resid):
#     return np.sqrt(np.mean(np.square(resid)))

# def score_model(model, y, x):
#     pred = model.predict(x)
#     resid = pd.Series(y - pred, index=y.index, name='Residuals')
    
#     print('RMSE: {:.3f}'.format(rmse(resid)))
#     print('RMSE: {:.2f} for target > 0'.format(rmse(resid[y != 0])))
#     for p in (50, 70, 80, 90, 95, 99):
#         v = np.percentile(y, p)
#         print('RMSE: {:.3f} for target > {} (top {} percentile)'.format(rmse(resid[y>v]), v, 100-p))

# def plot_model(model, y=None, x=None):
        
#     if x is None:
#         x = x_train
#     if y is None:
#         y = y_train
        
#     fig, axs = sns.plt.subplots(2, 2, figsize=(14, 10))
#     axs = axs.flatten()
    
#     pred = model.predict(x)
#     resid = pd.Series(y - pred, index=y.index, name='Residuals')
    
#     resid.hist(bins=40, ax=axs[0]); axs[0].set_xlabel('Residuals')
#     sm.qqplot(resid, line='q', ax=axs[1]);  axs[1].set_xlabel('Residuals')
#     tbpd.hist2d(resid, pred, ax=axs[2],
#                 vlabel='Residuals', hlabel='Predicted value', 
#                 integer_aligned_bins=True)
#     tbpd.hist2d(y, pred, ax=axs[3],
#                 vlabel='True value', hlabel='Predicted value', 
#                 integer_aligned_bins=True, sqrt=True)
#     fig.tight_layout()

# Models

In [ ]:
def reduce_until_significant(y, x):
    drops = []
    rmses = []
    pb = tbiter.IProgressBar(range(len(x.columns)))
    for i in pb:
        res = sm.OLS(y, x).fit()
        e = uem.rmse(res.resid)
        rmses.append(e)
        if res.pvalues.max() > .05:
            c = res.pvalues.argmax()
            pb.set_state('RMSE={:.2f} p={:.2f} "{}"'.format(e, res.pvalues[c], c))
            drops.append(c)
            x = x.drop(c, axis=1)
        else:
            pb.set_state('Finished.')
            pb.finish()
            print('Stopping at {} drops'.format(len(drops)))
            break
    print(res.summary())
    return res, x

In [ ]:
def reduce_eval(cols=None, add_intercept=True, remove_mean=True, divide_std=True, x=None):
    with tbdev.Notify('Reduction done'):
        if x is None:
            x = x_train[cols]
        if remove_mean:
            x = x - x.mean()
        if divide_std:
            x = x/x.std()
        if add_intercept:
            x['intercept'] = 1

        model, x = reduce_until_significant(y_train, x)
        print('\n\n\n')
        uem.score_model(model, y_train, x)
        uem.plot_model(model, y_train, x)

### Full model 

Reduced until no insignificant terms remain

In [ ]:
reduce_eval(x_train.columns)

# L=5 min

In [ ]:
reduce_eval([c for c in x_train if 'L=5 min' in c])

## L=10 min

In [ ]:
reduce_eval([c for c in x_train if 'L=10 min' in c])

## L=20 min

In [ ]:
reduce_eval([c for c in x_train if 'L=20 min' in c])

## L=30 min

In [ ]:
reduce_eval([c for c in x_train if 'L=30 min' in c])

# L$\ge$20 min

In [ ]:
reduce_eval([c for c in x_train if 'L=30 min' in c or 'L=20 min' in c])

# L$\ge$10 min

In [ ]:
reduce_eval([c for c in x_train if 'L=30 min' in c or 'L=20 min' in c or 'L=10 min' in c])

Try with squares and stuff?

# L=5, squares

In [ ]:
tmpx = x_train[[c for c in x_train if 'L=5' in c]].copy()

tmpx[[c+'**2' for c in tmpx]] = np.square(tmpx)

In [ ]:
reduce_eval(x=tmpx)

# L=5, squares, no -deltaC, deltaC

In [ ]:
tmpx = x_train[[c for c in x_train if 'L=5' in c and 'Δ-C' not in c and 'ΔC' not in c]].copy()
tmpx[[c+'**2' for c in tmpx]] = np.square(tmpx)

reduce_eval(x=tmpx)

# L=5, sqrt + standard, no -deltaC, deltaC, no w=1, no w=10

In [ ]:
tmpx = x_train[[c for c in x_train if 'L=5' in c and 'Δ-C' not in c and 'ΔC' not in c\
                and not 'W=1' in c]].copy()
tmpx[['sqrt({})'.format(c) for c in tmpx]] = np.sqrt(tmpx)

# R2 = .643
# RMSE 2.532
reduce_eval(x=tmpx)

In [ ]:
tbpd = tbdev.reload(tbpd)

ax = tbpd.hist2d([1,2,3,3], [-1, 1,2,7], hlabel='h', vlabel='v', integer_aligned_bins=True, edgecolors='k')


# Plan

1. Done: Built and reduced full model for all lag >= unique(lag) 
    - Reduce it until no more insigificant terms remain. Expect it to be the 5-min terms.
2. Build models (and reduce them) using larger and larger minimum lags.

## History

- Tried with variables up to 5 minutes before. Looked good.
- Tried with 30 minutes before only. Looked really bad. Made an attempt with ANOVA. Still bad.
- Tried 30 min with RF. Even worse.

In [ ]:
TIMES = ('day', 'hour', 'minute')

def rmse(residuals):
    return np.sqrt(np.mean(np.square(residuals)))

def score_model(model, y, x):
    pred = model.predict(x)
    resid = pd.Series(y - pred, index=y.index, name='Residuals')
    
    print('RMSE: {:.2f}'.format(rmse(resid)))
    print('Non-zero RMSE: {:.2f}'.format(rmse(resid[y != 0])))

def plot_model(model, y, x):
    if hasattr(model, 'params'):
        n_features = len(model.params)
    else:
        n_features = model.n_features_
        
    n_rows = (len(TIMES) + 3 + n_features + 1) // 2 
    fig, axs = sns.plt.subplots(n_rows,2, figsize=(14, n_rows*3.5))
    axs = axs.flatten()
    
    pred = model.predict(x)
    resid = pd.Series(y - pred, index=y.index, name='Residuals')
    
    resid.hist(bins=40, ax=axs[0]); axs[0].set_xlabel('Residuals')
    sm.qqplot(resid, line='q', ax=axs[1]);  axs[1].set_xlabel('Residuals')
    tbpd.hist2d(resid, pred, ax=axs[2],
                vlabel='Residuals', hlabel='Predicted value', 
                integer_aligned_bins=True)

    N = 3
    for c in x:
        tbpd.hist2d(resid, x[c], ax=axs[N], 
                    vlabel='Residuals', integer_aligned_bins=True)
        N += 1

    for v in TIMES:
        tbpd.hist2d(resid, getattr(resid.index, v), 
                    ax=axs[N], vlabel='Residuals', 
                    hlabel=v, integer_aligned_bins=True)
        N += 1
        
    fig.tight_layout()